# Laboratoire 6: Réseaux de neurones récurrents

## Références

http://colah.github.io/posts/2015-08-Understanding-LSTMs/

https://distill.pub/2016/augmented-rnns/

https://github.com/mila-udem/ecolehiver2018/blob/master/Tutoriaux/Hiver18_0308_tutorial_rnn_lstm_solution.ipynb

### jean.philippe.reid@rd.mila.quebec

### Mars 2018


In [ ]:
%matplotlib inline

from deeplib.mila import display_logo
display_logo()

## Module utilitaire

Premièrement, installons pytorch et certains modules nécessaires pour compléter ce tutoriel. 

In [ ]:
from random import randint
import numpy as np
from time import time

import torch
import torch.nn as nn
import torch.utils.data as data_utils
from torch import optim
torch.backends.cudnn.version()

cuda = True

torch.manual_seed(123)

In [ ]:
def normalized(x, y):
    '''Cette fonction normalize un le tenseur 'x'. '''
    x = (x - torch.min(x,0)[0].view(1,x.shape[1])) / ((torch.max(x,0)[0] - torch.min(x,0)[0]).view(1,x.shape[1]))

    return x, x.sum(dim=1)

def standardized(x, y):
    '''  Cette fonction standardize un le tenseur 'x'. '''  
    x = (x - torch.mean(x, 0).view(1,x.shape[1])) / torch.std(x, 0).view(1,x.shape[1])

    return x, x.sum(dim=1)

def data_set(N, T, interval):
    ''' 
    inputs : 
    N : nombre de données 
    T : longueur de la séquence
    interval : l'intervalle dans lequel les nombres seront tirés pour générer les séquences. 
    
    outputs: retourne les séquences (xx) et les cibles (yy), en format torch.tensor 
    '''
    
    x = (torch.Tensor(N, T).random_(0, 2 * interval[1]) + interval[0])
   
    return x, x.sum(dim=1)
  
def print_sequence(x, y):
    '''
    x : Une séquence particulière, i.e dim(x) = 1 x T
    y : Cible liée à cette même séquence, i.e. dim(y) = 1
    retourne une série de caractères illustrant la séquence 
    dans un format convivial. 
    '''
    n = x.shape[0]
    for i, x_ in enumerate(x):
        if i == 0 : 
            string=' ' + str(x_)
        else: 
            string=string + ' + '+str(x_)

    return string+' = ' + str(y)
  
def adjust_lr(optimizer, lr0, epoch, total_epochs):
    '''
    Cette fonction diminue le taux d'apprentissage suivant une fonction 
    exponentielle avec le nombre d'époque. 
    
    optimizer: e.g. optim.SGD(... )
    lr0 : taux d'apprentissage initial
    epoch : époque à laquelle la mise à jour est effectuée. 
    total epochs: nombre d'époque totale
    
    exemple : 
    
    new_learning_rate = adjust_lr(optimizer, 0.01, e_, 100)
    
    '''
    
    lr = lr0 * (0.36 ** (epoch / float(total_epochs)))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr

def batch_loss(data_batch,model): 
    '''
    Cette fonction calcule le loss d'un jeu de donnée divisé en plusieurs   
    batchs. C'est nécessaire pour traiter de grandes quantités de données.     
    '''
    loss = 0
    n = 0
    with torch.no_grad():
        for batch_idx, (x, y) in enumerate(data_batch):
            x, y = model.input_format(x, y)
            out = model(x)
            loss += model.criterion(out, y)
            n += 1
        return loss.item() / n


def adjust_fontsize(ax):
    '''
    Par déformation professionnelle, j'ai le souci de préparer de belles figures. 
    Cette fonction est un clin d'oeil pour mon ancien superviseur de thèse.
    '''
    for ax in ax:
        for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] 
                 + ax.get_xticklabels() + ax.get_yticklabels()):
            item.set_fontsize(14)

## Objectif

L'objectif de ce tutoriel est de construire un modèle capable d'additionner ou soustraire une série de nombres. Ce jeu de données est facile à générer et nous permet de tester la capacité de plusieurs algorithmes tels que,

* RNN, LSTM 
* MLP


Le jeu de données est constitué d'une séquence de nombre de longueurs *seq_len* , à laquelle une cible est associée. Dans l'exemple ci-dessous, la i$^{eme}$ composante des données est explicitement détaillée (voir la Fig. 1). 

\begin{align}  
\mathrm x^{(i)} &= \left[ 4,-1,15,24\right], \mathrm x^{(i)} \in \mathbb R^{d_0} \\ 
\mathrm y^{(i)} &= 42, \mathrm y^{(i)} \in \mathbb R 
\end{align}

Il est important de noter que chaque composante du vecteur $\mathrm x^{(i)}$, peut être de plusieurs dimensions, c'est-à-dire $x^{(i)}_j \in \mathbb R^{d_1}$ où $d_1 > 1$. 


Ce jeu de données peut ainsi servir à entrainer un modèle de réseau de neurones récurrents (RNN), tel que le _long short term memory_(LSTM). Dans ce cas, chaque nombre sera l'entrée d'une couche cachée (_hidden layer_) de dimension $h_d$. 

Comme la cible est un nombre réel, il est nécessaire de rajouter une couche linéaire au modèle pour "ajuster" les dimensions (voir la Fig.2). Cette notion sera expliquée à l'aide d'un exemple détaillé ci-bas. 

![Texte alternatif…](https://github.com/jphreid/tutorial_ivado/raw/master/lstm-figures.001.jpeg)

Fig.1 : Jeu de données considéré. 

![Texte alternatif…](https://github.com/jphreid/tutorial_ivado/raw/master/lstm-figures.002.jpeg)


Fig.2 : Schéma d'un réseau récurrent. 

## Réseau de neurones récurrent (LSTM)

### Générer une couche LSTM
### lstm = nn.LSTM(input_size, hidden_size, num_layers)

Une couche LSTM peut incorporer plusieurs paramètres dont certains régis par le jeu de données (*input_size*), mais aussi d'autres paramètres essentiels pour optimiser la capacité du modèle, tels que *hidden_size*, *num_layers*, etc.

### Entrées d'une couche LSTM (*inputs*)
____ = <font color='red'>lstm</font>(*input*, (h0,c0))__

En plus des données (_input_), il est aussi possible d'initialiser les tenseurs h_0, c_0 définis comme les *hidden* et *cell states*,  paramètres essentiels aux LSTMs. 

__Dans le cas où $h_0$ et $c_0$ ne sont pas définis, le module LSTM utilisera les valeurs par défaut, i.e. 0.__

### Donnée d'entrée (*input*)
__<font color='red'>input</font> =  torch.Tensor(seq_len, batch_size, input_size)__

Il est nécessaire de réorganiser les données d'entrées (input) selon trois paramètres : 

* la longueur de la séquence (seq_len)
* la grandeur du lot (batch, i.e. batch_size)
* les dimensions des entrées (input_size)

 


### Donnée de sortie (*output*)
### output = lstm(*input*, (h0,c0))
__<font color='red'>output</font> = torch.tensor(seq_len,batch_size, hidden_size x num_directions)__


Les dimensions du tenseur "output_0" sont déterminées par: (*seq_len*, *batch*, *hidden_size* $\times$ *num_directions*). 

Dans le cas qui nous intéresse, num_directions $ = 1$. 

![Texte alternatif…](https://github.com/jphreid/tutorial_ivado/raw/master/lstm-figures.003.jpeg)

Fig. 3: http://pytorch.org/docs/master/nn.html?highlight=lstm#torch.nn.LSTM

![Texte alternatif…](https://github.com/jphreid/tutorial_ivado/raw/master/lstm-figures.004.jpeg)

Fig. 4: http://pytorch.org/docs/master/nn.html?highlight=lstm#torch.nn.LSTM

![Texte alternatif…](https://github.com/jphreid/tutorial_ivado/raw/master/lstm-figures.005.jpeg)

Fig.5 : http://pytorch.org/docs/master/nn.html?highlight=lstm#torch.nn.LSTM

# Exemple

Créons un jeu de données $\bf x$ composés de 100 séquences de 4 nombres entre 0 et 100. Les cibles $\bf y$ correspondent à la somme de chacune de ces séquences. La fonction *data_set* située dans la section *Module utilitaire* s'occupe de cette tâche fastidieuse. 



In [ ]:
# Création d'un jeu de donnés 
x,y = data_set(100, 4, [-100, 100])

print(x.shape)
print(y.shape)

In [ ]:
sequence = print_sequence(x[1,:] ,y[1])

print(sequence)
print('Dimensions des entrées : {} x {}'.format(*x.shape))

__Q1__ : Selon l'exemple ci-haut, quels sont les paramètres d'entrées? On considéra ici qu'un seul lot (*batch*). 

* batch_size = 
* seq_len = 
* input = 



In [ ]:
# RÉPONSE: 
x_ = x[:,:,np.newaxis]
y_ = y[:,np.newaxis].type(torch.FloatTensor)

# ...

print('Dimensions du tenseur x = {}'.format(x_.shape))
print('______________________________________________')
print('    ')

print('Plus en détails: ')
print('_________________')
print('    ')

dimensions = x_.shape
print('batch_size = {}   seq_len = {}     input_size = {}'.format(*dimensions))

__Q2__  : Donnez un exemple où les dimensions d'entrées sont supérieures à un, c.-à-d. *input* $> 1$.

__Q3__  : Construisez votre première couche LSTM. Nous vous référons au lien dessous la Fig. 3 pour plus de détails et exemples. 

In [ ]:
# Q3
# les paramètres
input_size = 1
hidden_size = 6
num_layers = 1

# lstm
#lstm = ...

# forward 
output_0, (hn, cn) = lstm(x_)

print('Dimensions - output_0: {}'.format(output_0.shape))
print('Dimensions - h_n: {}'.format(hn.shape))
print('Dimensions - c_n: {}'.format(cn.shape))

### Réorganiser les dimensions de la sortie ("<font color='red'>output_0</font>") 

Tel que mentionné ci-haut, seule la dernière composante du tenseur "output_0" sera considérée pour prédire la cible associée à une séquence (voir Fig.2). 

Exemple: considérons la $41^e$ séquence ainsi que la couche associée. 

In [ ]:
print(x_[41,:,:])
print(y_[41])

sequence = print_sequence(x[41,:],y[41])
print(sequence)

In [ ]:
print(output_0[41,:,:])

Tel que mentionné ci-haut, seule la quatrième (dernière) composante du tenseur output_0 qui doit être considéré (voir Fig. 2).

In [ ]:
output_0[41,-1,:]

Le tenseur $h_n$ regroupe la dernière sortie (*output*) de chaque séquence.

In [ ]:
print(hn[:,41,:])

__Q4__  : Proposez une stratégie pour transformer le tenseur "output_0" de dimensions [100,4,6] à un tenseur de dimensions voulues, c.-à-d. [100,1]. Spécifiquement, quelle opération mathématique permettra d'obtenir les dimensions voulues?

In [ ]:
## Question

'''
Première partie: générez un tenseur M de dimensions voulues et utilisez 
la fonction 'torch.matmul' pour  multiplier M et output_0. 

M = torch.Tensor( _?_, _?_ )
output = torch.matmul(output_0[_?_, _?_, _?_], M)
'''


'''
Deuxième partie: gérérez une couche linéaire de dimensions voulues avec la 
fonction 'LL = nn.Linear(_?_, _?_)'. 

L'entrée de LL sera output_0, i.e. output = LL(output_0[_?_, _?_, _?_])
'''


print(output.shape)

### Construction du modèle

Il est fortement suggéré d'utiliser des classes pour définir les modèles dans l'environnement PyTorch, et ce peu importe son architecture. 


__Q5__  : Écrivez la fonction RnnLinear.forward(). Cette fonction aura comme entrée *self* et le tenseur $\bf x$ de dimension N x T x 1. 

Indice : le output_0 de la couche LSTM ou RNN sera l'entrée de la couche linéaire. 




In [ ]:
class RnnLinear(nn.Module):
    def __init__(self,input_size, hidden_size, num_layers, lstm_or_rnn,cuda):
        '''
        Les paramètres importants du modèle sont définies dans cette fonction. 
        '''
        num_directions = 1
        super(RnnLinear, self).__init__()
        
        # parametres importants 
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm_or_rnn = lstm_or_rnn      
        
        # couches de notre réseau 
        if lstm_or_rnn == 'lstm':
            self.lstm = nn.LSTM(input_size, hidden_size, 
                                num_layers,dropout = 0.5,
                                batch_first = True)
            if cuda : self.lstm.cuda() 
              
        elif lstm_or_rnn == 'rnn':
            self.rnn = nn.RNN(input_size, hidden_size,
                              num_layers,dropout = 0.5, 
                              batch_first = True
                             )
            if cuda: self.rnn.cuda()
              
        else: print('You made a mistake, pal!')
        
        self.linear = nn.Linear(hidden_size, input_size,bias=True)
        if cuda: 
            self.linear.cuda()
              
        # criteria for SGD
        self.criterion = nn.MSELoss()
        
    def input_format(self,x,y):
        '''
        Cette fonction permet de préparer les dimensions des données d'entrées. 
        '''
        x = x[:,:,None]
        y = y[:,None].type(torch.FloatTensor)
        
        if cuda:
            x = x.cuda()
            y = y.cuda()
        return x,y

    def grad_norm(self):
        total_norm=0
        for p in list(self.parameters()):
            param_norm = p.grad.data.norm(2)
            total_norm += param_norm.item()

        return total_norm
  
    def forward(self,x):
        '''
        Cette fonction devrait inclure des conditions if/elif sur la variable 
        self.lstm_or_rnn. La sortie des couches LSTM et RNN seront les entrées 
        de la couche linéaire. 
        '''
        
        # cas où self.lstm_or_rnn = 'rnn'
        if self.lstm_or_rnn == 'rnn':
            # out1,(__) = ...
            pass

        # cas où self.lstm_or_rnn = 'lstm'      
        if self.lstm_or_rnn == 'lstm':
            # out1,(__) = ...
            pass

        # votre couche linéraire             
        # out2 = ...

        return out2

Testons notre modèle!

Prenez le temps de bien comprendre les dimensions affichées ci-bas. 

In [ ]:
x,y = data_set(100, 4, [0,100])


input_size = 1
num_layers = 2
hidden_size = 40 

model = RnnLinear(input_size, hidden_size, num_layers, 'lstm', cuda)

xx,yy = model.input_format(x, y)
print('Dimensions initiales des données : {}'.format(x.shape))
print('Dimensions des données formatées : {}'.format(xx.shape))

pred = model(xx)
print('Dimensions des prédictions : {}'.format(pred.shape))

### Questions

__Q6__

Entrainez le modèle sur 20 époques avec ces paramètres et hyperparamètres, c.-à-dire $N_{train}$ = 20000, T = 10, intervalle = [0,1000], $h_d$ = 20, num_layers = 2, lr0 = 0.05. 

__Q7__ : 

Testez le modèle sur l'ensemble de test. Commentez les résultats obtenus.  

Note: L1 loss est définie comme la somme des différences absolue entre les cibles et les prédictions. Pour l'ensemble de test considéré, cette quantité est définie comme \begin{align}
L1 = \sum_{i=1}^{5000} \mid y^{(i)} - \text{output}^{(i)}\mid \end{align}

__Q8__ 

Finalement, *'standardisez'* les données de chaque séquence. Vous devrez utiliser la fonction *x, y = standardized(x, y)* définie dans le module utilitaire. 

__Q9__

Augmenter le nombre d'epoch à 100. Observez l'apprentissage.

__Q10__

Il est possible d'activer ou de désactiver l'option dropout de notre modèle en appelant 'model.train()' et 'model.eval()' respectivement. Testez ces deux modes. 

Pourquoi est-il nécessaire de désactiver l'option 'dropout' lors des phases test et validation? 

__Q11__ 

Explorez différents hyperparamètres. 

* lr = 0.1, 1, 0.00001

* hidden_size = 100,150, 200

* dropout = 0.1, 0.3, 0.5

Enjoy! 



### Création du jeu de données

In [ ]:
x,y = data_set(20000, 10, [0,1000])

#  standardized
xtrain, ytrain = (x[:10000], y[:10000])
xvalid, yvalid = (x[10000:15000], y[10000:15000])
xtest, ytest = (x[15000:], y[15000:])

batch_size = 100

all_data_train = data_utils.DataLoader(data_utils.TensorDataset(xtrain, ytrain),
                                       batch_size, shuffle=True)
all_data_valid = data_utils.DataLoader(data_utils.TensorDataset(xvalid, yvalid), 
                                       batch_size, shuffle=False)
all_data_test = data_utils.DataLoader(data_utils.TensorDataset(xtest, ytest),
                                       batch_size, shuffle=False)

### Entraînement du modèle

In [ ]:
input_size = 1
num_layers = 2
hidden_size = 20
number_epoch = 20
lr0 = 0.05

model = RnnLinear(input_size, hidden_size, num_layers,'lstm',cuda)
optimizer = optim.SGD(model.parameters(),lr=lr0)

# tableau à remplir lors de la phase d'entrainement 
loss_train_data = np.array([]) 
loss_valid_data = np.array([]) 

total_norm = np.array([])

t0 = time()  

for e_ in range(number_epoch):
  
    # model.train(), poruquoi? Indice : dropout   
    model.train()
    for batch_idx, (xx, yy) in enumerate(all_data_train):                
        # Formatez les données
        xx,yy = model.input_format(xx,yy)

        # Évaluez le modèle 
        pred_batch = model(xx)

        # Initialisez les gradients
        optimizer.zero_grad()

        # Calculez la loss  
        loss_batch = model.criterion(pred_batch,yy)

        # SGD backward 
        loss_batch.backward()

        # SGD optimized
        optimizer.step()

    # stop. 
  
    # Calculez la norm de tous les gradients 
    total_norm = np.append(total_norm,model.grad_norm())

    # model.eval(), pourquoi? Indice : dropout 
    model.eval()  

    loss_train_data = np.append(loss_train_data, batch_loss(all_data_train,model)) 
    loss_valid_data = np.append(loss_valid_data, batch_loss(all_data_valid,model))

    if e_%10 == 0: 
        # '{} {}'.format('foo', 'bar')
        print('N. of Epochs = {}, Train Loss = {}, Valid Loss = {}'.format(e_, loss_train_data[e_], loss_valid_data[e_]))


    lr_ = adjust_lr(optimizer, lr0, e_, number_epoch)

tf = time()
print('Terminé, {} sec'.format(tf-t0))

### Prédiction du modèle

In [ ]:
# prêt pour évaluer
model.eval()

# prêt pour entraîner
# model.train()

xtest_,ytest_ = model.input_format(xtest, ytest)
pred_test_all = model(xtest_)

L1_loss = torch.sum(torch.abs(pred_test_all - ytest_)).item()

print('LSTM L1 loss = {}'.format(L1_loss))
print('      ')
print('Prediction')
print('___________________')
print(pred_test_all[0:10])
print('      ')
print('Target')
print('___________________')
print(ytest_[:10])

### Courbes d'apprentissage

Nous traçons plus bas deux graphiques d'importance capitale : 

* Courbe d'apprentissage, i.e. loss vs. époque pour l'ensemble de validation et d'entrainement. 
* La norme de tous les gradients évaluée à chaque époque. 


In [ ]:
import matplotlib.pyplot as plt

fig, (ax1,ax2) = plt.subplots(1,2,figsize=(15,6))
ax1.plot(loss_train_data,'-k',label='Train')
ax1.plot(loss_valid_data,'-r',label='Valid')

ax2.plot(total_norm,'-b',label='Grad')


ax1.set_xlabel('epoch',fontsize=14)
ax2.set_xlabel('epoch',fontsize=14)
ax1.set_ylabel('MSE loss',fontsize=14)	
ax2.set_ylabel('Grad Norm',fontsize=14)	
legend = ax1.legend(loc='upper right',fontsize=14)

ax1.set_xlim((0))

ax1.set_title('Learning curve', fontsize = 14)
ax1.set_title('Learning curve', fontsize = 14)

xmin, xmax = ax2.get_xlim()
ymin, ymax = ax2.get_ylim()

ax2.text(0.6*(xmax - xmin) + xmin, 0.8*(ymax-ymin) + ymin,
         'num_layers : '+str(num_layers), fontsize=14)

ax2.text(0.6*(xmax - xmin) + xmin, 0.72*(ymax-ymin) + ymin,
         'hidden_size : '+str(hidden_size), fontsize=14)

ax2.text(0.6*(xmax - xmin) + xmin, 0.64*(ymax-ymin) + ymin,
         'number_epoch : '+str(number_epoch), fontsize=14)

ax2.text(0.6*(xmax - xmin) + xmin, 0.56*(ymax-ymin) + ymin,
         'lr$_0$ : '+str(lr0), fontsize=14)


adjust_fontsize([ax1,ax2])         

### Question

__Q12__ : Dans le graphique ci-haut, comment expliquez-vous que la norme diminue et converge vers zéro avec le nombre d'époques? 

...

## Prédiction sur des séquences de longueurs différentes

### Question

__Q13__ : Prouvez qu'il est aussi possible d'utiliser le modèle entrainé pour prédire des séquences d'une longueur différente.

In [ ]:
T_= 12

xx, yy = data_set(500,T_,[0,100])
xx, yy = normalized(xx, yy)

xx, yy = model.input_format(xx, yy)

print('Séquence de longueur {}'.format(T_))
print('--------------------------')
print('Prédiction: {}'.format(model(xx).data[0][0]))

print('Cible: {}'.format(yy.data[0][0]))
print('LSTM L1 loss: {}'.format((torch.abs(model(xx).data[0]-yy.data[0])[0])))

# Comparaison entre RNN et LSTM


__Q14__ : Entrainez le modèle ci-bas pour ces séries d'hyperparamètres et comparez les performances des modèles RNN et LSTM. 

* 'Stardardized' les données. 
* N = 30000
* $h_d$ = 50
* number_epoch = 100
* batch_size = 100
* lr0 = 0.005

  * T = 20, 50, 100, 200
  





In [ ]:
cuda = True 

input_size = 1
num_layers = 2
hidden_size = 50
number_epoch = 100
batch_size = 100
lr0 = 0.005

loss_train_data_rnn = np.array([])
loss_train_data_lstm = np.array([])

loss_valid_data_rnn = np.array([])
loss_valid_data_lstm = np.array([])

total_norm_rnn = np.array([])
total_norm_lstm = np.array([])

# Génération des données
T = 100

x,y = data_set(30000, T, [-10000,10000])

xtrain, ytrain = normalized(x[:20000], y[:20000])
xvalid, yvalid = normalized(x[20000:25000], y[20000:25000])
xtest, ytest = normalized(x[25000:], y[25000:])

batch_size = 100

all_data_train = data_utils.DataLoader(data_utils.TensorDataset
                          (xtrain, ytrain),batch_size, shuffle=True)
  
all_data_valid = data_utils.DataLoader(data_utils.TensorDataset
                          (xvalid, yvalid),batch_size, shuffle=False)

all_data_test = data_utils.DataLoader(data_utils.TensorDataset
                          (xtest, ytest),batch_size, shuffle=False)

# Initialisation des modèeles
model_rnn = RnnLinear(input_size, hidden_size, num_layers, 'rnn', cuda)
model_lstm = RnnLinear(input_size, hidden_size, num_layers, 'lstm', cuda)
optimizer_rnn = optim.SGD(model_rnn.parameters(), lr=lr0)
optimizer_lstm = optim.SGD(model_lstm.parameters(), lr=lr0)

t0 = time()  
for e_ in range(number_epoch):  
    model_rnn.train()
    
    ##################
    ###### RNN ######
    ##################
    
    for batch_idx, (xx, yy) in enumerate(all_data_train):                
        xx, yy = model_rnn.input_format(xx, yy)
        pred_batch_rnn = model_rnn(xx)
        optimizer_rnn.zero_grad()
        loss_batch_rnn = model_rnn.criterion(pred_batch_rnn, yy)
        loss_batch_rnn.backward()
        optimizer_rnn.step()

    model_rnn.eval()
    lt = batch_loss(all_data_train, model_rnn)
    lv = batch_loss(all_data_valid, model_rnn)
    loss_train_data_rnn = np.append(loss_train_data_rnn, lt)
    loss_valid_data_rnn = np.append(loss_valid_data_rnn, lv)

    total_norm_rnn = np.append(total_norm_rnn,model_rnn.grad_norm())
    
    ##################
    ###### LSTM ######
    ##################

    model_lstm.train()
    for batch_idx, (xx, yy) in enumerate(all_data_train):
        xx,yy = model_lstm.input_format(xx, yy)
        pred_batch_lstm = model_lstm(xx)
        optimizer_lstm.zero_grad()
        loss_batch_lstm = model_lstm.criterion(pred_batch_lstm,yy)
        loss_batch_lstm.backward()
        optimizer_lstm.step()

    
    model_lstm.eval()
    lt = batch_loss(all_data_train, model_lstm)
    lv = batch_loss(all_data_valid, model_lstm)
    loss_train_data_lstm = np.append(loss_train_data_lstm, lt)
    loss_valid_data_lstm = np.append(loss_valid_data_lstm, lv)

    total_norm_lstm = np.append(total_norm_lstm, model_lstm.grad_norm())  

    ##################
    ##################
    
    if e_%10 == 0: 
        print('N. of Epochs # {}, RNN, Train loss = {}  ----    LSTM, Train loss = {}'
                .format(e_,loss_train_data_rnn[e_],loss_train_data_lstm[e_]))
    
    lr_ = adjust_lr(optimizer_rnn,lr0, e_, number_epoch)
    lr_ = adjust_lr(optimizer_lstm,lr0, e_, number_epoch)

    tf = time()
print('Terminé, %.1f sec'%(tf - t0))

In [ ]:
fig, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2,2,figsize=(16,12))
ax1.plot(loss_train_data_rnn, '-k', label ='train')
ax1.plot(loss_valid_data_rnn, '-r', label = 'valid')

ax2.plot(total_norm_rnn, '-k', label = 'norm rnn')

ax3.plot(loss_train_data_lstm, '-k', label ='train')
ax3.plot(loss_valid_data_lstm, '-r', label = 'valid')

ax4.plot(total_norm_lstm, '-k', label = 'norm lstm')

ax3.set_xlabel('epoch', fontsize=14)
ax4.set_xlabel('epoch', fontsize=14)

ax1.set_ylabel('MSE loss', fontsize=14)	
ax3.set_ylabel('MSE loss', fontsize=14)	

ax1.set_title('RNN', fontsize = 14)
ax2.set_title('RNN', fontsize = 14)
ax3.set_title('LSTM', fontsize = 14)
ax4.set_title('LSTM', fontsize = 14)
legend = ax1.legend(loc='upper right', fontsize=14)
legend = ax3.legend(loc='upper right', fontsize=14)


ax1.set_xlim((0,number_epoch))
ax1.set_ylim((0))
ax2.set_ylim((0))
ax3.set_xlim((0,number_epoch))
ax3.set_ylim((0))
ax4.set_ylim((0))

xmin, xmax = ax1.get_xlim()
ymin, ymax = ax1.get_ylim()

ax1.text(0.1*(xmax - xmin) + xmin, 0.6*(ymax-ymin) + ymin,
         'num_layers : %.f'%num_layers, fontsize=14)
ax1.text(0.1*(xmax - xmin) + xmin, 0.52*(ymax-ymin) + ymin,
         'hidden_size : %.f'%hidden_size, fontsize=14)
ax1.text(0.1*(xmax - xmin) + xmin, 0.44*(ymax-ymin) + ymin,
         'number_epoch : %.f'%number_epoch, fontsize=14)
ax1.text(0.1*(xmax - xmin) + xmin, 0.36*(ymax-ymin) + ymin,
         'lr$_0$ : %.1f'%lr0, fontsize=14)


for ax in [ax1,ax2,ax3,ax4]:
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] 
                   + ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(14)        